# RLSS2023 - DQN Tutorial: Fitted Q Iteration (FQI)

Website: https://rlsummerschool.com/

Github repository: https://github.com/araffin/rlss23-dqn-tutorial

Gymnasium documentation: https://gymnasium.farama.org/

## Introduction

In this notebook, you will implement the Fitted Q Iteration( (FQI) algorithm to solve the [CartPole](https://gymnasium.farama.org/environments/classic_control/cart_pole/) problem.

This notebooks will first cover the basics for using the Gymnasium library: how to instantiate an environment, step into it and collect training data from the FQI algorithm.

You will then learn how to implement step-by-step the FQI algorithm which is the predecessor of the [Deep Q-Network (DQN)](https://stable-baselines3.readthedocs.io/en/master/modules/dqn.html) algorithm.

In [ ]:
# for autoformatting
# !pip install jupyter-black
# %load_ext jupyter_black

## Install Dependencies

In [ ]:
!pip install git+https://github.com/araffin/rlss23-dqn-tutorial/ --upgrade

In [ ]:
!apt-get install ffmpeg  # For visualization

## First steps with the Gym interface

An environment that follows the [gym interface](https://gymnasium.farama.org/) is quite simple to use.
It provides to this user mainly three methods, which have the following signature (for gym versions > 0.26):

- `reset()` called at the beginning of an episode, it returns an observation and a dictionary with additional info (defaults to an empty dict)
- `step(action)` called to take an action with the environment, it returns the next observation, the immediate reward, whether new state is a terminal state (episode is finished), whether the max number of timesteps is reached (episode is artificially finished), and additional information
- (Optional) `render()` which allow to visualize the agent in action. Note that graphical interface does not work on google colab, so we cannot use it directly (we have to rely on `render_mode='rbg_array'` to retrieve an image of the scene).

Under the hood, it also contains two useful properties:
- `observation_space` which one of the gym spaces (`Discrete`, `Box`, ...) and describe the type and shape of the observation
- `action_space` which is also a gym space object that describes the action space, so the type of action that can be taken

The best way to learn about [gym spaces](https://gymnasium.farama.org/api/spaces/) is to look at the [source code](https://github.com/Farama-Foundation/Gymnasium/tree/main/gymnasium/spaces), but you need to know at least the main ones:
- `gym.spaces.Box`: A (possibly unbounded) box in $R^n$. Specifically, a Box represents the Cartesian product of n closed intervals. Each interval has the form of one of [a, b], (-oo, b], [a, oo), or (-oo, oo). Example: A 1D-Vector or an image observation can be described with the Box space.
```python
# Example for using image as input:
observation_space = spaces.Box(low=0, high=255, shape=(HEIGHT, WIDTH, N_CHANNELS), dtype=np.uint8)
```                                       

- `gym.spaces.Discrete`: A discrete space in $\{ 0, 1, \dots, n-1 \}$
  Example: if you have two actions ("left" and "right") you can represent your action space using `Discrete(2)`, the first action will be 0 and the second 1.

## CartPole Environment

For this example, we will use CartPole environment, a classic control problem.

"A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The system is controlled by applying a force of +1 or -1 to the cart. The pendulum starts upright, and the goal is to prevent it from falling over. A reward of +1 is provided for every timestep that the pole remains upright. "

Cartpole environment: [https://gymnasium.farama.org/environments/classic_control/cart_pole/](https://gymnasium.farama.org/environments/classic_control/cart_pole/)

![Cartpole](https://cdn-images-1.medium.com/max/1143/1*h4WTQNVIsvMXJTCpXm_TAw.gif)

In [ ]:
import gymnasium as gym

# Instantiate the environment
env = ...

In [ ]:
# Box(4,) means that it is a Vector with 4 components
print("Observation space:", env.observation_space)
print("Shape:", env.observation_space.shape)
# Discrete(2) means that there is two discrete actions
print("Action space:", env.action_space)

In [ ]:
# The reset method is called at the beginning of an episode


In [ ]:
# Sample a random action

print(f"Sampled action: {action}")

In [ ]:
# step in the environment


In [ ]:
# Note the obs is a numpy array
# info is an empty dict for now but can contain any debugging info
# reward is a scalar
print(obs.shape, reward, terminated, truncated, info)

### Exercise (10 minutes): write the function to collect data

This function collects an offline dataset of transitions that will be used to train a model using the FQI algorithm.

See docstring of the function for what is expected as input/output.

In [ ]:
from dataclasses import dataclass

import numpy as np
from gymnasium import spaces


@dataclass
class OfflineData:
    """
    A class to store transitions.
    """

    observations: np.ndarray  # same as "state" in the theory
    next_observations: np.ndarray
    actions: np.ndarray
    rewards: np.ndarray
    terminateds: np.ndarray

**HINT**: Take a look at the slide on Gym API or at the Gymnasium documentation: https://gymnasium.farama.org/

In [ ]:
def collect_data(env_id: str, n_steps: int = 50_000) -> OfflineData:
    """
    Collect transitions using a random agent (sample action randomly).

    :param env_id: The name of the environment.
    :param n_steps: Number of steps to perform in the env.
    :return: The collected transitions.
    """
    # Create the Gym env
    env = gym.make(env_id)

    assert isinstance(env.observation_space, spaces.Box)
    # Numpy arrays (buffers) to collect the data
    observations = np.zeros((n_steps, *env.observation_space.shape))
    next_observations = np.zeros((n_steps, *env.observation_space.shape))
    # Discrete actions
    actions = np.zeros((n_steps, 1))
    rewards = np.zeros((n_steps,))
    terminateds = np.zeros((n_steps,))

    # Variable to know if the episode is over (done = terminated or truncated)
    done = False

    ### YOUR CODE HERE
    # You need to collect transitions for `n_steps` using
    # a random agent (sample action uniformly).
    # Do not forget to reset the environment if the current episode is over
    # (done = terminated or truncated)
    #
    # TODO:
    # 1. Sample a random action
    # 2. Step in the env using this random action
    # 3. Retrieve the new transition data (observation, reward, ...)
    #  and update the numpy arrays (buffers)
    # 4. Repeat until you collected `n_steps` transitions

    # Start the first episode
    current_obs, _ = env.reset()
    
    for idx in range(n_steps):
        # Sample a random action

        # Step in the environment

        # Store the transition
        # Note: we only record true termination (timeouts/truncations are artificial terminations)

        
        # Update current observation

        # Check if the episode is over

        # Don't forget to reset the env at the end of an episode


    ### END OF YOUR CODE

    return OfflineData(
        observations,
        next_observations,
        actions,
        rewards,
        terminateds,
    )

Let's try the collect data method:

In [ ]:
env_id = "CartPole-v1"
n_steps = 50_000
# Collect transitions for n_steps
data = collect_data(env_id=env_id, n_steps=n_steps)

In [ ]:
# Check the length of the collected data
assert len(data.observations) == n_steps
assert len(data.actions) == n_steps
# Check that there are multiple episodes in the data
assert not np.all(data.terminateds)
assert np.any(data.terminateds)
# Check the shape of the collected data
if env_id == "CartPole-v1":
    assert data.observations.shape == (n_steps, 4)
    assert data.next_observations.shape == (n_steps, 4)
assert data.actions.shape == (n_steps, 1)
assert data.rewards.shape == (n_steps,)

In [ ]:
from pathlib import Path

from dqn_tutorial.fqi import save_data

output_filename = Path("../data") / f"{env_id}_data"
# Create folder if it doesn't exist
output_filename.parent.mkdir(parents=True, exist_ok=True)

# Save collected data using numpy
save_data(data, output_filename)

## Fitted Q Iteration (FQI) Algorithm

Fitted Q Iteration (FQI) is an algorithm that extends q-learning to continuous state space using function estimators.

It iteratively approximates the q-values for a given dataset of transitions (state, action, reward, next_state) by iteratively solving a regression problem.

Compared to later algorithms like DQN, FQI uses the whole dataset at every iteration (working on the whole batch instead of using minibatches).


<div>
    <img src="https://araffin.github.io/slides/dqn-tutorial/images/fqi/tabular_limit_2.png" width="500"/>
</div>

In [ ]:
from functools import partial
from pathlib import Path
from typing import Optional

import gymnasium as gym
import numpy as np
from gymnasium import spaces
from sklearn import tree
from sklearn.base import RegressorMixin
from sklearn.exceptions import NotFittedError
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor

### Choosing a Model

With FQI, you can use any regression model.

Here we are choosing a [k-nearest neighbors regressor](https://scikit-learn.org/stable/modules/neighbors.html#regression), but one could choose a linear model, a decision tree, a neural network, ...

In [ ]:
# First choose the regressor
model_class = partial(KNeighborsRegressor, n_neighbors=30)  # LinearRegression, GradientBoostingRegressor

### Loading offline dataset

The offline dataset contains the transitions collected using a random agent.

In [ ]:
from dqn_tutorial.fqi import load_data

env_id = "CartPole-v1"
output_filename = Path("../data") / f"{env_id}_data.npz"
render_mode = "rgb_array"

# Create test environment
env = gym.make(env_id, render_mode=render_mode)

# Load saved transitions
data = load_data(output_filename)

### First Iteration of FQI

For $n = 0$, the initial training set is defined as:

- $x = (s_t, a_t)$
- $y = r_t$

We fit a regression model $f_\theta(x) = y$ to obtain $ Q^{n=0}_\theta(s, a) $

In [ ]:
# First iteration:
# The target q-value is the reward obtained
targets = data.rewards.copy()
# Create input for current observations and actions
# Concatenate the observations and actions
# so we can predict qf(s_t, a_t)
current_obs_input = np.concatenate((data.observations, data.actions), axis=1)
# Fit the estimator for the current target
model = model_class().fit(current_obs_input, targets)

### 1. Exercise (10 minutes): write the function to predict Q-Values


<div>
    <img src="https://araffin.github.io/slides/dqn-tutorial/images/fqi/q_value_fqi.png" width="300"/>
</div>

**HINT**: Check the first iteration of FQI to know how to create the input to the regression model


**HINT**: For efficiency, we are not working with one transition at a time but on a batch of transitions/observations. The resulting `q_values` are therefore a matrix of shape `(batch_size, n_actions)` where `batch_size` is the size of the batch of observations (aka number of observations used at once) and `n_actions` is the number of available actions (`n_actions=2` for CartPole, as we can only go left or right)

Below is a diagram explaining how the predicted `q_values` will be stored.

Row-wise, we get the q-values estimates for all possible actions but for one single observation (state):

<div>
    <img src="https://araffin.github.io/slides/dqn-tutorial/images/sklearn/q_value_batch.png" width="500"/>
    <br>
</div>

Column-wise, we get the q-values estimates for a given action but for a batch of observations (states):
<div>
    <img src="https://araffin.github.io/slides/dqn-tutorial/images/sklearn/q_value_batch_2.png" width="400"/>
</div>

In [ ]:
def get_q_values(
    model: RegressorMixin,
    obs: np.ndarray,
    n_actions: int,
) -> np.ndarray:
    """
    Retrieve the q-values for a set of observations(=states in the theory).
    qf(s_t, action) for all possible actions.

    :param model: Q-value estimator
    :param obs: A batch of observations
    :param n_actions: Number of discrete actions.
    :return: The predicted q-values for the given observations
        (batch_size, n_actions)
    """
    batch_size = len(obs)
    q_values = np.zeros((batch_size, n_actions))

    ### YOUR CODE HERE
    # TODO: for every possible actions a:
    # 1. Create the regression model input $(s, a)$ for the action a
    # and states s (here a batch of observations)
    # 2. Predict the q-values for the batch of states
    # 3. Update q-values array for the current action a

    # Predict q-value for each action
    for action_idx in range(n_actions):
        # Note: we should do one hot encoding if not using CartPole (n_actions > 2)
        # Create a vector of size (batch_size, 1) for the current action
        # This allows to do batch prediction for all the provided observations
        actions = action_idx * np.ones((batch_size, 1))
        # Concatenate the observations and the actions to obtain
        # the input to the q-value estimator
        # you can use `np.concatenate()`

        # Predict q-values for the given observation/action combination
        # shape: (batch_size, 1)
        # with a scikit-learn model, you can `model.predict()`

        # Update the q-values array for the current action
        # q_values is of shape (batch_size, n_actions)
        

    ### END OF YOUR CODE

    return q_values

Let's test it with a subset of the collected data:

In [ ]:
n_observations = 2
n_actions = int(env.action_space.n)

q_values = get_q_values(model, data.observations[:n_observations], n_actions)

assert q_values.shape == (n_observations, n_actions)

### 2. Exercise (8 minutes): write the function to evaluate a model

A greedy policy $\pi(s)$ can be defined using the q-value:

$\pi(s) = argmax_{a \in A} Q(s, a)$.

It is the policy that takes the action with the highest q-value for a given state.

In [ ]:
import os

from gymnasium.wrappers.monitoring.video_recorder import VideoRecorder


def evaluate(
    model: RegressorMixin,
    env: gym.Env,
    n_eval_episodes: int = 10,
    video_name: Optional[str] = None,
) -> None:
    episode_returns, episode_reward = [], 0.0
    total_episodes = 0
    done = False

    # Setup video recorder
    video_recorder = None
    if video_name is not None and env.render_mode == "rgb_array":
        os.makedirs("../logs/videos/", exist_ok=True)

        video_recorder = VideoRecorder(
            env=env,
            base_path=f"../logs/videos/{video_name}",
        )

    current_obs, _ = env.reset()
    # Number of discrete actions
    n_actions = int(env.action_space.n)
    assert isinstance(env.action_space, spaces.Discrete), "FQI only support discrete actions"

    while total_episodes < n_eval_episodes:
        # Record video
        if video_recorder is not None:
            video_recorder.capture_frame()

        ### YOUR CODE HERE

        # Retrieve the q-values for the current observation
        # you need to re-use `get_q_values()`
        # Note: you need to add a batch dimension to the observation
        # you can use `current_obs[np.newaxis, ...]` for that: (obs_dim,) -> (batch_size=1, obs_dim)

        # Select the action that maximizes the q-value for each state
        # Don't forget to remove the batch dimension, you can use `.item()` for that

        # Send the action to the env and update current observation

        ### END OF YOUR CODE

        episode_reward += float(reward)

        done = terminated or truncated
        if done:
            episode_returns.append(episode_reward)
            episode_reward = 0.0
            total_episodes += 1
            current_obs, _ = env.reset()

    if video_recorder is not None:
        print(f"Saving video to {video_recorder.path}")
        video_recorder.close()

    print(f"Total reward = {np.mean(episode_returns):.2f} +/- {np.std(episode_returns):.2f}")

In [ ]:
# Evaluate the first iteration
evaluate(model, env, n_eval_episodes=10)

### 3. Exercise (20 minutes): the Fitted Q Iterations

1. Create the training set based on the previous iteration $ Q^{n-1}_\theta(s, a) $ and the transitions:
- input: $x = (s_t, a_t)$
- if $s_{t+1}$ is non-terminal: $y = r_t + \gamma \cdot \max_{a' \in A}(Q^{n-1}_\theta(s_{t+1}, a'))$
- if $s_{t+1}$ is terminal, do not bootstrap: $y = r_t$

2. Fit a model $f_\theta$ using a regression algorithm to obtain $ Q^{n}_\theta(s, a)$
 
\begin{aligned}
 f_\theta(x) = y
\end{aligned}

4. Repeat, $n = n + 1$

First, let's define some constants:

In [ ]:
# Max number of iterations
n_iterations = 20
# How often do we evaluate the learned model
eval_freq = 2
# How many episodes to evaluate every eval-freq
n_eval_episodes = 10
# discount factor
gamma = 0.99
# Number of discrete actions
n_actions = int(env.action_space.n)

Then do several iteration of the FQI algorithm

**HINT**: For that exercise, you should take a look at the first iteration of FQI.

**HINT**: The`data` variable (containing the offline dataset) is an instance of `OfflineData`:

```python
@dataclass
class OfflineData:
    """
    A class to store transitions.
    """

    observations: np.ndarray  # same as "state" in the theory
    next_observations: np.ndarray
    actions: np.ndarray
    rewards: np.ndarray
    terminateds: np.ndarray
```

In [ ]:
# Load saved transitions
data = load_data(output_filename)

print(type(data))
print(data.observations.shape)

**HINT**: As for the first exercise, we are working here with a batch of data. Thanks to numpy, you can efficiently compute the max over a batch:

<div>
    <img src="https://araffin.github.io/slides/dqn-tutorial/images/sklearn/q_values_max.png" width="500"/>
</div>

**HINT**: Using masking, you can elegantly handle the two cases for the regression target. The value of $y$ depends on whether the episode is terminated or not (in one case we bootstrap using the q-value, in the other case, we do not bootstrap and use the terminal reward instead).

Visually, masking is an element-wise operation:

<div>
    <img src="https://araffin.github.io/slides/dqn-tutorial/images/sklearn/masking.png" width="500"/>
</div>



In [ ]:
for iter_idx in range(n_iterations):
    ### YOUR CODE HERE
    # TODO:
    # 1. Compute the q values for the next states using
    # the previous regression model
    # 2. Keep only the next q values that correspond
    # to the greedy-policy
    # 3. Construct the regression target (TD(0) target)
    # 4. Fit a new regression model with this new target

    # First, retrieve the q-values for the next states (data.next_observations)
    # for all possible actions
    # you need to use `get_q_values()` method

    # Follow-greedy policy: use the action with the highest q-value
    # to compute the next q-values

    # The new target is the reward + what our agent expect to get
    # if it follows a greedy policy (follow action with the highest q-value)
    # Reminder: you should not bootstrap if terminated=True
    # (you can mask the next q values for that using `np.logical_not`)


    # Update the q-value estimate for the current (observations, actions) pair
    # (current_obs_input)
    # with the current target,
    # i.e., fit a regression model using the new target
    # Note: you can use `model_class()` to instantiate a new model
    model = ...

    
    ### END OF YOUR CODE

    if (iter_idx + 1) % eval_freq == 0:
        print(f"Iter {iter_idx + 1}")
        print(f"Score: {model.score(current_obs_input, targets):.2f}")
        evaluate(model, env, n_eval_episodes)

### Record a video of the trained agent

In [ ]:
eval_env = gym.make(env_id, render_mode="rgb_array")
video_name = f"FQI_{env_id}"
n_eval_episodes = 3

evaluate(model, eval_env, n_eval_episodes, video_name=video_name)

In [ ]:
from dqn_tutorial.notebook_utils import show_videos

print(f"FQI agent on {env_id} after {n_iterations} iterations:")
show_videos("../logs/videos/", prefix=video_name)

### Going further

- play with different models, and with their hyperparameters
- play with the discount factor
- play with the number of data collected/used
- combine data from random policy with data from trained model

## Conclusion

What we have seen in this notebook:
- collecting data using a random agent in a gym environment
- predicting q-values using a regression model
- the fitted q-iteration (FQI) algorithm to learn from an offline dataset